# Results of the 2021 citizen's referendum "Deutsche Wohnen & Co. enteignen"

The inspiration from [Tagesspiegel](https://interaktiv.tagesspiegel.de/lab/wahlen-2021-berlin-so-haben-die-berliner-kieze-gewaehlt)

![](tagesspiegel_map.jpg)

In [1]:
import numpy as np
import pandas as pd
import json
import warnings; warnings.simplefilter('ignore')

In [2]:
import openpyxl
import itertools as it

def read_specific_sheet(fn, sheet_name='Tabelle1'):
    
    book = openpyxl.load_workbook(fn, data_only=True)
    print(book.sheetnames, sheet_name)
    #assert sheet_name in book.sheetnames 
    
    def row_is_good(row):
        #     return (
        #         not all([elem is None for elem in row]) 
        #         #and
        #         #row[0] != 'SUMME'
        #     )
        return True
    
    sheet_as_list = [
        row for row in it.islice(book[sheet_name].values, 0, None) if row_is_good(row)
    ]

    df = pd.DataFrame.from_records(
        data=sheet_as_list[1:],
        columns=sheet_as_list[0],
    )
    
    return df

def list_of_points(inp):

    def flatten_any_iterator(itr):
        for x in itr:
            try:
                yield from flatten_any_iterator(x)
            except TypeError:
                yield x
    
    flat_list = list(flatten_any_iterator(inp))
    
    return [[fst, snd] for fst, snd in zip(flat_list[0::2], flat_list[1::2])]

# Referendum results

Election results are publically published [here](https://wahlen-berlin.de/abstimmungen/VE2021/AFSPRAES/index.html)

* Read xlsx file, clean header
* Note the number of entries: Results are reported for each **Urnenwahlbezirk** and each **Briefwahlbezirk**
    * A Briefwahlbezirk can consist of one or more Urnenwahlbezirk
* Issue: Results from a Briefwahlbezirk cannot be reliably assigned to an Urnenwahlbezirk. Disregarding votes from the Briefwahlbezirke will lead to skewed election results, especially since the Briefwahl (mail-in voting) was exentsively used in 2021 due to the ongoing Covid pandemic
    * Trade-off: Briefwahlbezirk still offers sufficient geographical granularity. Assign Urnenwahlbezirk results to Briefwahlbezirke

In [3]:
referendum_results = read_specific_sheet(
    r'DL_BE_VE2021.xlsx',
    'VE_2021_mod'
).rename(
    columns = {
        'Adresse':'adr',
        'Stimmabgabe Volksentscheid: Ja' : 'ja_abs',
        'Ja! in %' : 'ja_rel',
        'Stimmabgabe Volksentscheid: Nein' : 'nein_abs', 
        'Nein in %' : 'nein_rel', 
        'Ungültige Stimmen': 'ung_abs',
        'Gültige Stimmen': 'glt_abs',
        'Abgeordneten-\nhauswahlkreis' : 'AWKs',
        'Bundestags-\nwahlkreis' : 'BWKs',
        
        # Wahlberechtigte
        'Wahlberechtigte insgesamt' : 'n_wbs_insg',
        'Wahlberechtigte A1' : 'n_wbs_im_wz_ohne_ws', 
        'Wahlberechtigte A2' : 'n_wbs_im_wz_mit_ws', 
        'Wahlberechtigte A3' : 'n_wbs_nicht_im_wz_ohne_ws',
        
        #        
        'Teilnehmende' : 'teiln', 
        'Teilnehmende B1' : 'teiln_bw', 
    }
).drop(
    columns=['aufn']
).astype(
    dtype={
        'adr' : 'string', 
        'Stimmart' : 'string', 
        'Bezirksnummer' : 'string',
        'Bezirksname' : 'string', 
        'Wahlbezirk' : 'string',
        'Wahlbezirksart' : 'string', 
        'Briefwahlbezirk' : 'string', 
        'AWKs' : 'string',
        'BWKs' : 'string',
        'OstWest' : 'string', 
        'n_wbs_insg' : 'float',
        'n_wbs_im_wz_ohne_ws' : 'float', 
        'n_wbs_im_wz_mit_ws' : 'float', 
        'n_wbs_nicht_im_wz_ohne_ws' : 'float',
        'teiln' : 'float', 
        'teiln_bw' : 'float', 
        'ung_abs' : 'float',
        'glt_abs' : 'float', 
        'ja_abs' : 'float',
        'ja_rel' : 'float',
        'nein_abs' : 'float', 
        'nein_rel' : 'float', 
    }
).set_index( 'adr', drop=False)

referendum_results

['Impressum', 'Erläuterungen', 'VE_2021_mod'] VE_2021_mod


,Stimmart,adr,ID,Bezirksnummer,Bezirksname,Wahlbezirk,Wahlbezirksart,Briefwahlbezirk,AWKs,BWKs,...,n_wbs_im_wz_mit_ws,n_wbs_nicht_im_wz_ohne_ws,teiln,teiln_bw,ung_abs,glt_abs,ja_abs,ja_rel,nein_abs,nein_rel
adr,,,,,,,,,,,,,,,,,,,,,
01W100,Stimme,01W100,01100,01,Mitte,100,W,011A,01,75,...,741.0,0.0,519.0,12.0,11.0,508.0,299.0,58.858268,209.0,41.141732
01W101,Stimme,01W101,01101,01,Mitte,101,W,011B,01,75,...,490.0,0.0,366.0,10.0,6.0,360.0,182.0,50.555556,178.0,49.444444
01W102,Stimme,01W102,01102,01,Mitte,102,W,011B,01,75,...,692.0,0.0,406.0,10.0,8.0,398.0,217.0,54.522613,181.0,45.477387
01W103,Stimme,01W103,01103,01,Mitte,103,W,011C,01,75,...,569.0,0.0,329.0,10.0,9.0,320.0,159.0,49.687500,161.0,50.312500
01W104,Stimme,01W104,01104,01,Mitte,104,W,011D,01,75,...,639.0,0.0,522.0,17.0,10.0,512.0,263.0,51.367188,249.0,48.632812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12B6S,Stimme,12B6S,12B6S,12,Reinickendorf,6S,B,126S,06,77,...,0.0,0.0,761.0,761.0,12.0,749.0,264.0,35.246996,485.0,64.753004
12B6T,Stimme,12B6T,12B6T,12,Reinickendorf,6T,B,126T,06,77,...,0.0,0.0,449.0,449.0,7.0,442.0,127.0,28.733032,315.0,71.266968
12B6U,Stimme,12B6U,12B6U,12,Reinickendorf,6U,B,126U,06,77,...,0.0,0.0,399.0,399.0,4.0,395.0,171.0,43.291139,224.0,56.708861


Check number of Urnenwahlbezirke. Switch to Briefwahlbezirke by selecting `referendum_results.Wahlbezirksart == 'B'`

In [4]:
referendum_results.loc[referendum_results.Wahlbezirksart == 'W']

,Stimmart,adr,ID,Bezirksnummer,Bezirksname,Wahlbezirk,Wahlbezirksart,Briefwahlbezirk,AWKs,BWKs,...,n_wbs_im_wz_mit_ws,n_wbs_nicht_im_wz_ohne_ws,teiln,teiln_bw,ung_abs,glt_abs,ja_abs,ja_rel,nein_abs,nein_rel
adr,,,,,,,,,,,,,,,,,,,,,
01W100,Stimme,01W100,01100,01,Mitte,100,W,011A,01,75,...,741.0,0.0,519.0,12.0,11.0,508.0,299.0,58.858268,209.0,41.141732
01W101,Stimme,01W101,01101,01,Mitte,101,W,011B,01,75,...,490.0,0.0,366.0,10.0,6.0,360.0,182.0,50.555556,178.0,49.444444
01W102,Stimme,01W102,01102,01,Mitte,102,W,011B,01,75,...,692.0,0.0,406.0,10.0,8.0,398.0,217.0,54.522613,181.0,45.477387
01W103,Stimme,01W103,01103,01,Mitte,103,W,011C,01,75,...,569.0,0.0,329.0,10.0,9.0,320.0,159.0,49.687500,161.0,50.312500
01W104,Stimme,01W104,01104,01,Mitte,104,W,011D,01,75,...,639.0,0.0,522.0,17.0,10.0,512.0,263.0,51.367188,249.0,48.632812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12W622,Stimme,12W622,12622,12,Reinickendorf,622,W,126T,06,77,...,514.0,0.0,475.0,9.0,42.0,433.0,119.0,27.482679,314.0,72.517321
12W623,Stimme,12W623,12623,12,Reinickendorf,623,W,126S,06,77,...,441.0,0.0,391.0,13.0,7.0,384.0,124.0,32.291667,260.0,67.708333
12W624,Stimme,12W624,12624,12,Reinickendorf,624,W,126U,06,77,...,461.0,0.0,543.0,3.0,22.0,521.0,229.0,43.953935,292.0,56.046065


# Urnenwahlbezirk, Briefwahlbezirke and their geometries

Urnenwahlbezirke are the smallest geometric units in which election results are known. They typically enclose a few blocks and can be represented drawing polygons on a map

* Public source [Open Data Berlin](https://daten.berlin.de/datensaetze/geometrien-der-wahllokale-f%C3%BCr-die-wahlen-zum-deutschen-bundestag-berlin-und-zum)
* Adaption of provided geometry file in maplibre-compatible format
    * Use [QGIS](https://qgis.org/en/site/) to convert existing combination of .shp/.dbf files to .geojson
    * Unite Urnenwahlbezirk-polygons according to the Briefwahlbezirk in which they are contained (QGIS functionality)
    * Important pitfall: Preserve the correct coordinate system (EPSG:4326-WGS 84)
* Need to assign Urnenwahlbezirk to the correct Briefwahlbezirke `uwbs_in_bwbs`
    * Use 2 geojson files: 
        * uwbs_4326_geojson_adresse: Urnenwahlbezirk
        * bwbs_4326_geojson_simplified14: Briefwahlbezirk
            * This file results from the QGIS pre-processing by uniting Urnenwahlbezirk to a single Briefwahlbezirk if necessary

In [5]:
with open(r'polygons_uwbs.json', 'r', encoding='utf-8') as fh:
    geojson = json.loads(fh.read())

In [6]:
geojson.keys()

dict_keys(['type', 'name', 'crs', 'features'])

In [7]:
#geojson['features'][0]
len(geojson['features'])

2257

In [8]:
geojson['features'][0]['properties']

{'UWB': '01100',
 'BWB': '011A',
 'AWK': '0101',
 'BEZ': '01',
 'BWK': '75',
 'Adresse': '01W100',
 'NAME_BEZ': 'Mitte',
 'NAME_AWK': 'Mitte 1'}

In [9]:
geojson_properties = pd.DataFrame.from_records([
    dict(
        adr=feat['properties']['Adresse'],
        bid=feat['properties']['BEZ'],
        bez=feat['properties']['NAME_BEZ'],
        awk=feat['properties']['AWK'][2:],
        bwb=feat['properties']['BWB'],
        uwb=feat['properties']['UWB'],
    ) for feat in geojson['features']
]).set_index('adr', drop=False)

geojson_properties

,adr,bid,bez,awk,bwb,uwb
adr,,,,,,
01W100,01W100,01,Mitte,01,011A,01100
01W101,01W101,01,Mitte,01,011B,01101
01W102,01W102,01,Mitte,01,011B,01102
01W103,01W103,01,Mitte,01,011C,01103
01W104,01W104,01,Mitte,01,011D,01104
...,...,...,...,...,...,...
12W622,12W622,12,Reinickendorf,06,126T,12622
12W623,12W623,12,Reinickendorf,06,126S,12623
12W624,12W624,12,Reinickendorf,06,126U,12624


In [10]:
uwbs_in_bwbs = geojson_properties.groupby('bwb').uwb.apply(lambda llist: sorted([elem[2:] for elem in llist])).sort_index()
uwbs_in_bwbs

bwb
011A    [100, 124]
011B    [101, 102]
011C    [103, 125]
011D    [104, 105]
011E    [106, 108]
           ...    
126S    [621, 623]
126T         [622]
126U         [624]
126V         [625]
126W         [626]
Name: uwb, Length: 1507, dtype: object

In [11]:
geojson_geometries = pd.DataFrame.from_records([
    dict(
        adr    = feat['properties']['Adresse'], 
        coords = list_of_points(feat['geometry']['coordinates'])
    ) for feat in geojson['features']
]).set_index('adr')

geojson_geometries

,coords
adr,
01W100,"[[13.37094449810375, 52.52691629213833], [13.3..."
01W101,"[[13.368964234816657, 52.53376664039617], [13...."
01W102,"[[13.368964234816657, 52.53376664039617], [13...."
01W103,"[[13.382176185645713, 52.53126164239199], [13...."
01W104,"[[13.38798217526706, 52.52936176988101], [13.3..."
...,...
12W622,"[[13.301866779641525, 52.60839789793542], [13...."
12W623,"[[13.309623520114918, 52.607863548078356], [13..."
12W624,"[[13.292108274966239, 52.60251336250379], [13...."


## Sanity check geojson completeness

Check for elections districts (Wahlbezirke) that occur in the geojson but not in the election results. None occur! ✔️

In [12]:
geojson_properties.loc[~geojson_properties.adr.isin(referendum_results.index)]

,adr,bid,bez,awk,bwb,uwb
adr,,,,,,


Second sanity check: Show election districts that occur in the election result but not in the geojson file of the Urnenwahlbezirke. Now all Briefwahlbezirke show up, as should be ✔️

In [13]:
# referendum_results.loc[~referendum_results.index.isin(geojson_properties.adr) & (referendum_results.Wahlbezirksart == 'W')]
referendum_results.loc[~referendum_results.index.isin(geojson_geometries.index)]

,Stimmart,adr,ID,Bezirksnummer,Bezirksname,Wahlbezirk,Wahlbezirksart,Briefwahlbezirk,AWKs,BWKs,...,n_wbs_im_wz_mit_ws,n_wbs_nicht_im_wz_ohne_ws,teiln,teiln_bw,ung_abs,glt_abs,ja_abs,ja_rel,nein_abs,nein_rel
adr,,,,,,,,,,,,,,,,,,,,,
01B1A,Stimme,01B1A,01B1A,01,Mitte,1A,B,011A,01,75,...,0.0,0.0,1078.0,1078.0,2.0,1076.0,565.0,52.509294,511.0,47.490706
01B1B,Stimme,01B1B,01B1B,01,Mitte,1B,B,011B,01,75,...,0.0,0.0,1061.0,1061.0,13.0,1048.0,458.0,43.702290,590.0,56.297710
01B1C,Stimme,01B1C,01B1C,01,Mitte,1C,B,011C,01,75,...,0.0,0.0,1177.0,1177.0,11.0,1166.0,617.0,52.915952,549.0,47.084048
01B1D,Stimme,01B1D,01B1D,01,Mitte,1D,B,011D,01,75,...,0.0,0.0,961.0,961.0,5.0,956.0,513.0,53.661088,443.0,46.338912
01B1E,Stimme,01B1E,01B1E,01,Mitte,1E,B,011E,01,75,...,0.0,0.0,1133.0,1133.0,12.0,1121.0,534.0,47.636039,587.0,52.363961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12B6S,Stimme,12B6S,12B6S,12,Reinickendorf,6S,B,126S,06,77,...,0.0,0.0,761.0,761.0,12.0,749.0,264.0,35.246996,485.0,64.753004
12B6T,Stimme,12B6T,12B6T,12,Reinickendorf,6T,B,126T,06,77,...,0.0,0.0,449.0,449.0,7.0,442.0,127.0,28.733032,315.0,71.266968
12B6U,Stimme,12B6U,12B6U,12,Reinickendorf,6U,B,126U,06,77,...,0.0,0.0,399.0,399.0,4.0,395.0,171.0,43.291139,224.0,56.708861


## Geo shapes of the Briefwahlbezirke

* Uniting the Urnenwahlbezirk-polygons into Briefwahlbezirk-polygons happened in QGIS

In [14]:
# Assign Bezirk-number to string representation (name of the Bezirk)

bez_dict = {
    '01' : 'Mitte',
    '02' : 'Friedrichshain-Kreuzberg',
    '03' : 'Pankow',
    '04' : 'Charlottenburg-Wilmersdorf',
    '05' : 'Spandau',
    '06' : 'Steglitz-Zehlendorf',
    '07' : 'Tempelhof-Schöneberg',
    '08' : 'Neukölln',
    '09' : 'Treptow-Köpenick',
    '10' : 'Marzahn-Hellersdorf',
    '11' : 'Lichtenberg',
    '12' : 'Reinickendorf',
}

In [15]:
with open(r'polygons_bwbs.json', 'r', encoding='utf-8') as fh:
    geojson_bwbs = json.loads(fh.read())

In [16]:
len(geojson_bwbs['features']) # Checks out! 1507 -> Number of Briefwahlbezirke

1507

In [17]:
geojson_bwbs['features'][0]['properties']

{'UWB': '08508', 'BWB': '085E', 'AWK': '0805', 'BEZ': '08', 'BWK': '82'}

Collect meta data (properties) of Briefwahlbezirk, most importantly which Urnenwahlbezirke belongs to them

In [18]:
geojson_properties_bwbs = pd.DataFrame.from_records([
    dict(
        bid=feat['properties']['BEZ'],
        awk=feat['properties']['AWK'][2:],
        bwb=feat['properties']['BWB'],
        bwk=feat['properties']['BWK'],
    ) for feat in geojson_bwbs['features']
]).set_index('bwb', drop=False)

geojson_properties_bwbs['bez'] = geojson_properties_bwbs.bid.apply(bez_dict.get)
geojson_properties_bwbs['awk'] = geojson_properties_bwbs[['bid', 'awk']].apply(lambda x: ''.join(x), axis=1)
geojson_properties_bwbs = geojson_properties_bwbs.join(uwbs_in_bwbs)

geojson_properties_bwbs.sort_index()

,bid,awk,bwb,bwk,bez,uwb
bwb,,,,,,
011A,01,0101,011A,75,Mitte,"[100, 124]"
011B,01,0101,011B,75,Mitte,"[101, 102]"
011C,01,0101,011C,75,Mitte,"[103, 125]"
011D,01,0101,011D,75,Mitte,"[104, 105]"
011E,01,0101,011E,75,Mitte,"[106, 108]"
...,...,...,...,...,...,...
126S,12,1206,126S,77,Reinickendorf,"[621, 623]"
126T,12,1206,126T,77,Reinickendorf,[622]
126U,12,1206,126U,77,Reinickendorf,[624]


Collect the actual polygon data

In [19]:
geojson_geometries_bwbs = pd.DataFrame.from_records([
    dict(
        bwb    = feat['properties']['BWB'], 
        coords = list_of_points(feat['geometry']['coordinates'])
    ) for feat in geojson_bwbs['features']
]).set_index('bwb')

geojson_geometries_bwbs

,coords
bwb,
085E,"[[13.441460916879349, 52.44542547191383], [13...."
085J,"[[13.429828479237852, 52.42587946943355], [13...."
085K,"[[13.40943749931206, 52.421213776728614], [13...."
085H,"[[13.429828479237852, 52.42587946943355], [13...."
085I,"[[13.443055221686587, 52.42665110692938], [13...."
...,...
085P,"[[13.42004694463059, 52.4127924318369], [13.41..."
085Q,"[[13.422050235894337, 52.41720975742624], [13...."
085F,"[[13.416258458864563, 52.444636365059516], [13..."


# Aggregate election results for each Briefwahlbezirk

In [20]:
referendum_results.keys()

Index(['Stimmart', 'adr', 'ID', 'Bezirksnummer', 'Bezirksname', 'Wahlbezirk',
       'Wahlbezirksart', 'Briefwahlbezirk', 'AWKs', 'BWKs', 'OstWest',
       'n_wbs_insg', 'n_wbs_im_wz_ohne_ws', 'n_wbs_im_wz_mit_ws',
       'n_wbs_nicht_im_wz_ohne_ws', 'teiln', 'teiln_bw', 'ung_abs', 'glt_abs',
       'ja_abs', 'ja_rel', 'nein_abs', 'nein_rel'],
      dtype='object')

In [21]:
# bwb_group.ID.apply(list).agg(sum)

In [22]:
aggregate_res = referendum_results.copy()
aggregate_res.adr = aggregate_res.adr.apply(lambda x: [x])

bwb_group = aggregate_res.groupby('Briefwahlbezirk')

results_bwb = bwb_group.agg({
    'Stimmart' : 'first', 
    'ID' : 'first', 
    'Bezirksnummer' : 'first', 
    'Bezirksname' : 'first', 
    'Wahlbezirk' : 'first',
    'Wahlbezirksart' : 'first',
    'Briefwahlbezirk' : 'first', 
    'AWKs' : 'first',
    'BWKs' : 'first', 
    'OstWest' : 'first',
       'n_wbs_insg': sum, 
    'n_wbs_im_wz_ohne_ws': sum, 
    'n_wbs_im_wz_mit_ws': sum,
       'n_wbs_nicht_im_wz_ohne_ws': sum,
    'teiln': sum, 
    'teiln_bw': sum,
       'ung_abs': sum, 
    'glt_abs': sum,
    'ja_abs': sum,
    'nein_abs': sum,
})

results_bwb['contains'] = bwb_group.adr.agg(sum)
results_bwb['n_distr' ] = results_bwb.contains.apply(len)

results_bwb['ja_rel'  ] = 100*results_bwb['ja_abs']/results_bwb['teiln']
results_bwb['nein_rel'] = 100*results_bwb['nein_abs']/results_bwb['teiln']
results_bwb['ung_rel' ] = 100*results_bwb['ung_abs']/results_bwb['teiln']
results_bwb['wahlbet' ] = 100*results_bwb['teiln']/results_bwb['n_wbs_insg']

#results_bwb.sort_values(by='n_distr', ascending=False)
results_bwb.sort_index(inplace=True)
results_bwb

,Stimmart,ID,Bezirksnummer,Bezirksname,Wahlbezirk,Wahlbezirksart,Briefwahlbezirk,AWKs,BWKs,OstWest,...,ung_abs,glt_abs,ja_abs,nein_abs,contains,n_distr,ja_rel,nein_rel,ung_rel,wahlbet
Briefwahlbezirk,,,,,,,,,,,,,,,,,,,,,
011A,Stimme,01100,01,Mitte,100,W,011A,01,75,W,...,26.0,2036.0,1167.0,869.0,"[01W100, 01W124, 01B1A]",3,56.595538,42.143550,1.260912,71.300138
011B,Stimme,01101,01,Mitte,101,W,011B,01,75,O,...,27.0,1806.0,857.0,949.0,"[01W101, 01W102, 01B1B]",3,46.753955,51.773050,1.472995,81.976744
011C,Stimme,01103,01,Mitte,103,W,011C,01,75,O,...,35.0,2024.0,1072.0,952.0,"[01W103, 01W125, 01B1C]",3,52.064109,46.236037,1.699854,81.771247
011D,Stimme,01104,01,Mitte,104,W,011D,01,75,O,...,21.0,1887.0,1049.0,838.0,"[01W104, 01W105, 01B1D]",3,54.979036,43.920335,1.100629,83.428072
011E,Stimme,01106,01,Mitte,106,W,011E,01,75,O,...,26.0,2006.0,968.0,1038.0,"[01W106, 01W108, 01B1E]",3,47.637795,51.082677,1.279528,82.101010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126S,Stimme,12621,12,Reinickendorf,621,W,126S,06,77,W,...,37.0,1544.0,554.0,990.0,"[12W621, 12W623, 12B6S]",3,35.041113,62.618596,2.340291,86.111111
126T,Stimme,12622,12,Reinickendorf,622,W,126T,06,77,W,...,49.0,875.0,246.0,629.0,"[12W622, 12B6T]",2,26.623377,68.073593,5.303030,85.714286
126U,Stimme,12624,12,Reinickendorf,624,W,126U,06,77,W,...,26.0,916.0,400.0,516.0,"[12W624, 12B6U]",2,42.462845,54.777070,2.760085,81.558442


In [23]:
# Reduce referendum data to absolute minimum
results_bwb_min = results_bwb[[
    'n_wbs_insg', 'teiln', 'teiln_bw', 
    'glt_abs', 'ung_abs', 'ung_rel', 'wahlbet',
    'ja_abs', 'ja_rel', 'nein_abs', 'nein_rel',
    'contains', 'n_distr'
]]
results_bwb_min

,n_wbs_insg,teiln,teiln_bw,glt_abs,ung_abs,ung_rel,wahlbet,ja_abs,ja_rel,nein_abs,nein_rel,contains,n_distr
Briefwahlbezirk,,,,,,,,,,,,,
011A,2892.0,2062.0,1096.0,2036.0,26.0,1.260912,71.300138,1167.0,56.595538,869.0,42.143550,"[01W100, 01W124, 01B1A]",3
011B,2236.0,1833.0,1081.0,1806.0,27.0,1.472995,81.976744,857.0,46.753955,949.0,51.773050,"[01W101, 01W102, 01B1B]",3
011C,2518.0,2059.0,1211.0,2024.0,35.0,1.699854,81.771247,1072.0,52.064109,952.0,46.236037,"[01W103, 01W125, 01B1C]",3
011D,2287.0,1908.0,989.0,1887.0,21.0,1.100629,83.428072,1049.0,54.979036,838.0,43.920335,"[01W104, 01W105, 01B1D]",3
011E,2475.0,2032.0,1147.0,2006.0,26.0,1.279528,82.101010,968.0,47.637795,1038.0,51.082677,"[01W106, 01W108, 01B1E]",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
126S,1836.0,1581.0,778.0,1544.0,37.0,2.340291,86.111111,554.0,35.041113,990.0,62.618596,"[12W621, 12W623, 12B6S]",3
126T,1078.0,924.0,458.0,875.0,49.0,5.303030,85.714286,246.0,26.623377,629.0,68.073593,"[12W622, 12B6T]",2
126U,1155.0,942.0,402.0,916.0,26.0,2.760085,81.558442,400.0,42.462845,516.0,54.777070,"[12W624, 12B6U]",2


# Housing data

In an act of citizen science, the activists behind the citizen's referendum where able to collect data on the number of houses that are either owned by a large real estate company or in collective ownership (Genossenschaft). Only houses in the first category are targeted for socialization. The acitvitsts were so kind to provide approximate numbers of houses for each Briefwahlbezirk.

In [24]:
houses_bwb = read_specific_sheet(
    r'Houses.xlsx',
    'Briefwahlbezirke'
).set_index('bwb_id')
houses_bwb

['Briefwahlbezirke'] Briefwahlbezirke


,bst_ente,bst_gssi
bwb_id,,
011A,4,3
011B,2,0
011C,8,4
011D,2,0
011F,3,4
...,...,...
123G,0,5
124J,0,13
124M,0,55


In [25]:
houses_bwb.sum(axis=0)

bst_ente    6923
bst_gssi    8885
dtype: int64

# Demografic data

Again this can be sourced from [Berlin Open Data](https://daten.berlin.de/) initiative

In [26]:
import openpyxl as excel
import pandas as pd
import itertools as it

In [27]:
xfile = excel.load_workbook(r'Demografic.xlsx', data_only=True)
sheet = xfile["Strukturdaten"]

top_row  = [elem for elem in next(sheet.values) if elem is not None]
left_col = [row[0] for row in list(sheet.values)[1:] if row[0] is not None]

first_col = 0
last_col  = len(top_row)
first_row = 1
last_row  = len(left_col)
print(first_col, last_col, first_row, last_row)

demographics = pd.DataFrame(
    data=(it.islice(row, first_col, last_col) for row in list(sheet.values)[first_row:last_row+1]),
    columns=top_row
).astype(
    dtype={
        'Adresse' : 'string', 
        'Bezirksnummer' : 'string', 
        'Bezirksname' : 'string', 
        'Wahlbezirk' : 'string',
        'Briefwahlbezirksnummer' : 'string', 
        'Ost/West' : 'string', 
        'Bundestagswahlkreis' : 'string',
        'Abgeordnetenhauswahlkreis' : 'string',
    }
)
demographics

0 62 1 2257


,Adresse,Bezirksnummer,Bezirksname,Wahlbezirk,Briefwahlbezirksnummer,Ost/West,Bundestagswahlkreis,Abgeordnetenhauswahlkreis,Einwohner Anzahl,Einwohner unter 6 Jahre Anzahl,...,Deutsche 18 + Familienstand ELP Prozent,Deutsche 18 + evangelisch Anzahl,Deutsche 18 + evangelisch Prozent,Deutsche 18 + katholisch Anzahl,Deutsche 18 + katholisch Prozent,Deutsche 18+ Veränderung zum Vorjahr,Deutsche 18+ Veränderung zum Vorjahr in Prozent,Einwohner unter 65 Jahre 2019 Anzahl,Einwohner unter 65 in SGB II 2019 Anzahl,Einwohner unter 65 in SGB II 2019 Prozent
0,01W100,01,Mitte,100,011A,W,75,01,3198,279,...,0.000000,203,13.875598,160,10.936432,440,43.010753,1876,348,18.550107
1,01W101,01,Mitte,101,011B,O,75,01,1724,141,...,0.614754,172,17.622951,143,14.651639,37,3.940362,1547,42,2.714932
2,01W102,01,Mitte,102,011B,O,75,01,2284,127,...,1.330203,235,18.388106,128,10.015649,-8,-0.622084,2066,79,3.823814
3,01W103,01,Mitte,103,011C,O,75,01,2275,241,...,0.589971,159,15.634218,154,15.142576,1,0.098425,2177,42,1.929260
4,01W104,01,Mitte,104,011D,O,75,01,2135,149,...,0.978793,260,21.207178,153,12.479608,-24,-1.920000,2122,43,2.026390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2252,12W622,12,Reinickendorf,622,126T,W,77,06,1399,73,...,0.000000,338,30.615942,136,12.318841,26,2.411874,941,22,2.337938
2253,12W623,12,Reinickendorf,623,126S,W,77,06,1167,66,...,0.000000,322,35.817575,109,12.124583,24,2.742857,780,11,1.410256
2254,12W624,12,Reinickendorf,624,126U,W,77,06,1395,70,...,0.000000,341,30.176991,164,14.513274,-29,-2.502157,861,47,5.458769
2255,12W625,12,Reinickendorf,625,126V,W,77,06,1323,46,...,0.268817,409,36.648746,67,6.003584,13,1.178604,922,46,4.989154


In [28]:
demographics.keys()

Index(['Adresse', 'Bezirksnummer', 'Bezirksname', 'Wahlbezirk',
       'Briefwahlbezirksnummer', 'Ost/West', 'Bundestagswahlkreis',
       'Abgeordnetenhauswahlkreis', 'Einwohner Anzahl',
       'Einwohner unter 6 Jahre Anzahl', 'Einwohner unter 6 Jahre Prozent',
       'Einwohner 6 - 18 Anzahl', 'Einwohner 6 - 18 Prozent',
       'Einwohner 18 - 65 Anzahl', 'Einwohner 18 - 65 Prozent',
       'Einwohner 65 und älter Anzahl', 'Einwohner 65 und älter Prozent',
       'Ausländer Anzahl', 'Ausländer Prozent', 'EU-Bürger 16 +',
       'EU-Bürger 16 + Prozent', 'Deutsche 16 +  Anzahl',
       'Deutsche 16 + Prozent', 'Deutsche 18 + Anzahl',
       'Deutsche 18 + Prozent', 'Deutsche 18 - 25 Anzahl',
       'Deutsche 18 - 25 Prozent', 'Deutsche 25 - 35 Anzahl',
       'Deutsche 25 - 35 Prozent', 'Deutsche 35 - 45 Anzahl',
       'Deutsche 35 - 45 Prozent', 'Deutsche 45 - 60 Anzahl',
       'Deutsche 45 - 60 Prozent', 'Deutsche 60 - 70 Anzahl',
       'Deutsche 60 - 70 Prozent', 'Deutsche 70 +

The glaring issue here is that the public data uses xenophobic categories such as 'Ausländer', which roughly translates to 'foreigner'. Ill-defined as it is, the category is not a sound basis for data analysis and will be excluded

In [29]:
these_please = [
    'Briefwahlbezirksnummer',
    # Einwohner
    'Einwohner Anzahl',
    'Einwohner unter 6 Jahre Anzahl',
    'Einwohner 6 - 18 Anzahl',
    'Einwohner 18 - 65 Anzahl',
    'Einwohner 65 und älter Anzahl',
    # Altersdemographie
    'Deutsche 18 - 25 Anzahl',
    'Deutsche 25 - 35 Anzahl',
    'Deutsche 35 - 45 Anzahl',
    'Deutsche 45 - 60 Anzahl',
    'Deutsche 60 - 70 Anzahl',
    'Deutsche 70 +  Anzahl',
    # Sozialhilfebezieher
    'Einwohner unter 65 in SGB II 2019 Anzahl',
    # Viel Zuzug?
    'Einwohner unter 65 Jahre 2019 Anzahl',
    'Deutsche 18 + Anzahl',
    'Deutsche 18+  Veränderung zum Vorjahr',  
]

In [30]:
#demographics[these_please].rename(columns={'Briefwahlbezirksnummer':'bwb'}).set_index('bwb')
demographics = demographics[these_please].rename(columns={'Briefwahlbezirksnummer':'bwb'}).groupby('bwb').sum()
demographics

,Einwohner Anzahl,Einwohner unter 6 Jahre Anzahl,Einwohner 6 - 18 Anzahl,Einwohner 18 - 65 Anzahl,Einwohner 65 und älter Anzahl,Deutsche 18 - 25 Anzahl,Deutsche 25 - 35 Anzahl,Deutsche 35 - 45 Anzahl,Deutsche 45 - 60 Anzahl,Deutsche 60 - 70 Anzahl,Deutsche 70 + Anzahl,Einwohner unter 65 in SGB II 2019 Anzahl,Einwohner unter 65 Jahre 2019 Anzahl,Deutsche 18 + Anzahl,Deutsche 18+ Veränderung zum Vorjahr
bwb,,,,,,,,,,,,,,,
011A,6156,502,462,4753,439,313,1001,557,445,197,214,777,4123,2727,500
011B,4008,268,302,3047,391,148,502,562,609,212,221,121,3613,2254,29
011C,5178,479,435,3998,266,161,800,708,569,142,158,238,4947,2538,8
011D,3889,240,410,2922,317,143,468,576,644,219,182,170,3761,2232,-37
011E,4236,262,285,3173,516,157,519,455,703,332,312,88,3711,2478,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126S,2346,120,304,1246,676,138,161,194,506,345,477,43,1622,1821,16
126T,1399,73,143,718,465,85,56,122,298,141,402,22,941,1104,26
126U,1395,70,130,668,527,42,84,120,265,195,424,47,861,1130,-29


## Compute percentages from absolute numbers

In [31]:
rentner_rel = demographics['Einwohner 65 und älter Anzahl']/demographics[[ 'Einwohner 18 - 65 Anzahl', 'Einwohner 65 und älter Anzahl',]].sum(axis=1)
old = 0.24 <= rentner_rel
print(np.count_nonzero(old), np.count_nonzero(old)/old.size, rentner_rel.mean())

793 0.5262110152621101 0.24027695191465


In [32]:
sgb_rel = demographics['Einwohner unter 65 in SGB II 2019 Anzahl']/demographics[[ 'Einwohner 18 - 65 Anzahl']].sum(axis=1)
sgb = 0.16918 <= sgb_rel
print(np.count_nonzero(sgb), np.count_nonzero(sgb)/sgb.size, sgb_rel.mean())

592 0.3928334439283344 0.16918557234957127


In [33]:
demo_selection = pd.DataFrame()
demo_selection['dem_rente'] = 100*demographics['Einwohner 65 und älter Anzahl']/demographics[[ 'Einwohner 18 - 65 Anzahl', 'Einwohner 65 und älter Anzahl']].sum(axis=1)
demo_selection['dem_sgb'  ] = 100*demographics['Einwohner unter 65 in SGB II 2019 Anzahl']/demographics[['Einwohner 18 - 65 Anzahl']].sum(axis=1)
# demo_selection['dem_sgb'  ] = 100*demographics['Einwohner unter 65 in SGB II 2019 Anzahl']/demographics[['Einwohner 18 - 65 Anzahl', 'Einwohner unter 6 Jahre Anzahl', 'Einwohner 6 - 18 Anzahl']].sum(axis=1)

demo_selection

,dem_rente,dem_sgb
bwb,,
011A,8.455316,16.347570
011B,11.372891,3.971119
011C,6.238274,5.952976
011D,9.786971,5.817933
011E,13.987530,2.773401
...,...,...
126S,35.171696,3.451043
126T,39.306847,3.064067
126U,44.100418,7.035928


In [34]:
demo_selection.describe()

,dem_rente,dem_sgb
count,1507.000000,1507.000000
mean,24.027695,16.918557
std,10.190020,12.784834
min,1.418440,0.000000
25%,16.181036,7.352270
50%,24.783446,13.291139
75%,31.022775,23.304133
max,62.705941,82.846371


# State government election results: The Abgeordnetenhaus data

Sourced from [here](https://wahlen-berlin.de/wahlen/BE2021/AFSPRAES/index.html).

We just want to look at the winner in the respective Briefwahlbezirk. The analysis is limited to the parties that actually reached sufficient votes to be admitted to the Abgeordnetenhaus. Two tables need to be evaluated due to the German voting system (Erst-, Zweitstimme)

In [35]:
import openpyxl as excel

In [36]:
def winner_agh(df, by='Briefwahlbezirk'):
    tmp = df.groupby(by).sum()

    parties = ['SPD', 'CDU', 'GRÜNE', 'DIE LINKE', 'AfD', 'FDP']
    for party in parties:
        tmp[party] = 100.0*tmp[party] / tmp['Gültige Stimmen']

    return tmp[parties].apply(lambda res: parties[np.argmax(res)], axis=1)

In [37]:
xfile = excel.load_workbook(r'DL_BE_AGHBVV2021.xlsx', data_only=True)
sheet = xfile["AGH_W1"]

top_row  = [elem for elem in next(sheet.values) if elem is not None]
left_col = [row[0] for row in list(sheet.values)[1:] if row[0] is not None]

first_col = 0
last_col  = len(top_row)
first_row = 1
last_row  = len(left_col)
print(first_col, last_col, first_row, last_row)

agh_erststimme = pd.DataFrame(
    data=(it.islice(row, first_col, last_col) for row in list(sheet.values)[first_row:last_row+1]),
    columns=top_row
)
agh_erststimme['agh_id'] = agh_erststimme['Bezirksname'] + ' ' + agh_erststimme['Abgeordneten-\nhauswahlkreis'].apply(lambda s: s[1:] if s[0]=='0' else s)
agh_erststimme

0 68 1 3764


,Stimmart,Adresse,Bezirksnummer,Bezirksname,Wahlbezirk,Wahlbezirksart,Briefwahlbezirk,Abgeordneten-\nhauswahlkreis,Bundestags-\nwahlkreis,OstWest,...,Kirschtowski,Pape,Sah,Snelinski,Wadehn,Weimer,Witte,Worbs,aufn,agh_id
0,Erststimme,01B1A,01,Mitte,1A,B,011A,01,75,W,...,0,0,0,0,0,0,0,0,None,Mitte 1
1,Erststimme,01B1B,01,Mitte,1B,B,011B,01,75,O,...,0,0,0,0,0,0,0,0,None,Mitte 1
2,Erststimme,01B1C,01,Mitte,1C,B,011C,01,75,O,...,0,0,0,0,0,0,0,0,None,Mitte 1
3,Erststimme,01B1D,01,Mitte,1D,B,011D,01,75,O,...,0,0,0,0,0,0,0,0,None,Mitte 1
4,Erststimme,01B1E,01,Mitte,1E,B,011E,01,75,O,...,0,0,0,0,0,0,0,0,None,Mitte 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3759,Erststimme,12W622,12,Reinickendorf,622,W,126T,06,77,W,...,0,0,0,0,0,0,0,0,None,Reinickendorf 6
3760,Erststimme,12W623,12,Reinickendorf,623,W,126S,06,77,W,...,0,0,0,0,0,0,0,0,None,Reinickendorf 6
3761,Erststimme,12W624,12,Reinickendorf,624,W,126U,06,77,W,...,0,0,0,0,0,0,0,0,None,Reinickendorf 6
3762,Erststimme,12W625,12,Reinickendorf,625,W,126V,06,77,W,...,0,0,0,0,0,0,0,0,None,Reinickendorf 6


In [38]:
sheet = xfile["AGH_W2"]

top_row  = [elem for elem in next(sheet.values) if elem is not None]
left_col = [row[0] for row in list(sheet.values)[1:] if row[0] is not None]

first_col = 0
last_col  = len(top_row)
first_row = 1
last_row  = len(left_col)
print(first_col, last_col, first_row, last_row)

agh_zweitstimme = pd.DataFrame(
    data=(it.islice(row, first_col, last_col) for row in list(sheet.values)[first_row:last_row+1]),
    columns=top_row
)
agh_zweitstimme['agh_id'] = agh_zweitstimme['Bezirksname'] + ' ' + agh_zweitstimme['Abgeordneten-\nhauswahlkreis'].apply(lambda s: s[1:] if s[0]=='0' else s)
agh_zweitstimme

0 53 1 3764


,Stimmart,Adresse,Bezirksnummer,Bezirksname,Wahlbezirk,Wahlbezirksart,Briefwahlbezirk,Abgeordneten-\nhauswahlkreis,Bundestags-\nwahlkreis,OstWest,...,du.,BÜNDNIS21,FREIE WÄHLER,Klimaliste Berlin,MIETERPARTEI,Die Humanisten,Team Todenhöfer,Volt,aufn,agh_id
0,Zweitstimme,01B1A,01,Mitte,1A,B,011A,01,75,W,...,2,1,5,12,4,5,16,37,None,Mitte 1
1,Zweitstimme,01B1B,01,Mitte,1B,B,011B,01,75,O,...,0,0,4,5,0,1,4,38,None,Mitte 1
2,Zweitstimme,01B1C,01,Mitte,1C,B,011C,01,75,O,...,4,0,9,7,4,4,3,45,None,Mitte 1
3,Zweitstimme,01B1D,01,Mitte,1D,B,011D,01,75,O,...,0,0,4,6,1,3,3,39,None,Mitte 1
4,Zweitstimme,01B1E,01,Mitte,1E,B,011E,01,75,O,...,1,0,8,7,0,2,6,34,None,Mitte 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3759,Zweitstimme,12W622,12,Reinickendorf,622,W,126T,06,77,W,...,0,0,2,1,0,3,0,2,None,Reinickendorf 6
3760,Zweitstimme,12W623,12,Reinickendorf,623,W,126S,06,77,W,...,0,0,1,0,0,0,2,6,None,Reinickendorf 6
3761,Zweitstimme,12W624,12,Reinickendorf,624,W,126U,06,77,W,...,1,0,9,2,0,0,3,3,None,Reinickendorf 6
3762,Zweitstimme,12W625,12,Reinickendorf,625,W,126V,06,77,W,...,0,0,6,1,0,0,3,4,None,Reinickendorf 6


In [39]:
agh_1 = winner_agh(agh_erststimme, 'Briefwahlbezirk')
agh_2 = winner_agh(agh_zweitstimme, 'Briefwahlbezirk')

agh = pd.concat([agh_1, agh_2], axis=1, keys=['Erststimme', 'Zweitstimme'])
agh

,Erststimme,Zweitstimme
Briefwahlbezirk,,
011A,GRÜNE,GRÜNE
011B,GRÜNE,GRÜNE
011C,GRÜNE,GRÜNE
011D,GRÜNE,GRÜNE
011E,GRÜNE,GRÜNE
...,...,...
126S,CDU,CDU
126T,CDU,CDU
126U,CDU,CDU


Easier formatting for the party names which is handy later on

In [47]:
short_party = {
    'SPD'       : 's',
    'CDU'       : 'c',
    'GRÜNE'     : 'g',
    'DIE LINKE' : 'l', 
    'AfD'       : 'a', 
    'FDP'       : 'f',
}

erststimmen_res = agh.Erststimme.apply(short_party.get)
erststimmen_res.name = 'agh_1'
# erststimmen_res

zweitstimmen_res = agh.Zweitstimme.apply(short_party.get)
zweitstimmen_res.name = 'agh_2'
# zweitstimmen_res

agh_12 = pd.concat([erststimmen_res, zweitstimmen_res], axis=1)
agh_12

,agh_1,agh_2
Briefwahlbezirk,,
011A,g,g
011B,g,g
011C,g,g
011D,g,g
011E,g,g
...,...,...
126S,c,c
126T,c,c
126U,c,c


In [48]:
np.count_nonzero(agh_12.agh_1 != agh_12.agh_2)

201

# Aggregate data & output geojson

All sources are collected in a single geojson file:

* geographic data, 
* referendum results
* housing data 
* election results
* demographic data

While the structure of geojson file is selfexplanatory, the process of constructing a large file can become confusing. Use some helper functions:

In [42]:
def new_geojson_feature(properties, list_of_points):
    return {
        'type': 'Feature',
        'properties': properties,
        'geometry': {
            'type': 'MultiPolygon',
            'coordinates': [[ list_of_points ]]
        }
    }

def build_features(properties, polygons):
    chk = [
        idx0 == idx1 for ((idx0, _), (idx1, _)) in zip(
            properties.sort_index().iterrows(),
            polygons.sort_index().iterrows(),
        )
    ]
    if not all(chk):
        raise(ValueError, 
              "{:d} Missmatching indexes".format(len(chk) - sum(chk)))
    return [
        new_geojson_feature(
            props.to_dict(), 
            polygon['coords']
        ) for ((_, props), (_, polygon)) in zip(
            properties.sort_index().iterrows(),
            polygons.sort_index().iterrows(),
        )
    ]
    
def new_geojson(name='default_name', features=[]):
    return {
        'type': 'FeatureCollection',
        'name': name,
        'crs': {
            'type': 'name',
            'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}
        },
        'features': features
    }

In [43]:
dset = geojson_properties_bwbs.join([
    results_bwb_min.drop(columns='contains'),
    houses_bwb,
    demo_selection,
    pd.concat([erststimmen_res, zweitstimmen_res], axis=1)
]).fillna(0)
dset.sort_index()

,bid,awk,bwb,bwk,bez,uwb,n_wbs_insg,teiln,teiln_bw,glt_abs,...,ja_rel,nein_abs,nein_rel,n_distr,bst_ente,bst_gssi,dem_rente,dem_sgb,agh_1,agh_2
bwb,,,,,,,,,,,,,,,,,,,,,
011A,01,0101,011A,75,Mitte,"[100, 124]",2892.0,2062.0,1096.0,2036.0,...,56.595538,869.0,42.143550,3,4.0,3.0,8.455316,16.347570,g,g
011B,01,0101,011B,75,Mitte,"[101, 102]",2236.0,1833.0,1081.0,1806.0,...,46.753955,949.0,51.773050,3,2.0,0.0,11.372891,3.971119,g,g
011C,01,0101,011C,75,Mitte,"[103, 125]",2518.0,2059.0,1211.0,2024.0,...,52.064109,952.0,46.236037,3,8.0,4.0,6.238274,5.952976,g,g
011D,01,0101,011D,75,Mitte,"[104, 105]",2287.0,1908.0,989.0,1887.0,...,54.979036,838.0,43.920335,3,2.0,0.0,9.786971,5.817933,g,g
011E,01,0101,011E,75,Mitte,"[106, 108]",2475.0,2032.0,1147.0,2006.0,...,47.637795,1038.0,51.082677,3,0.0,0.0,13.987530,2.773401,g,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126S,12,1206,126S,77,Reinickendorf,"[621, 623]",1836.0,1581.0,778.0,1544.0,...,35.041113,990.0,62.618596,3,0.0,0.0,35.171696,3.451043,c,c
126T,12,1206,126T,77,Reinickendorf,[622],1078.0,924.0,458.0,875.0,...,26.623377,629.0,68.073593,2,0.0,0.0,39.306847,3.064067,c,c
126U,12,1206,126U,77,Reinickendorf,[624],1155.0,942.0,402.0,916.0,...,42.462845,516.0,54.777070,2,0.0,0.0,44.100418,7.035928,c,c


In [44]:
geojson_new = new_geojson(
    name='uwbs',
    features=build_features(
        dset,
        geojson_geometries_bwbs
    )
)

In [45]:
with open(r'data.json', 'w', encoding='utf-8') as fh:
    fh.write(json.dumps(geojson_new))
    print('Done')

Done


Since excel files are easier to check the monolithic geojson, also output as a table

In [46]:
fn = r'data.xlsx'

with pd.ExcelWriter(fn) as fh:
    dset.to_excel(fh, sheet_name='Sheet1')